Загружаем исходные данные в colab.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset(1).csv to dataset(1).csv


Загружаем данные в датафрейм

In [ ]:
import io
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

df

,event_date,app_version,country_code,install_date,media_source,campaign_name,appsflyer_id,is_organic,payments,revenue_usd,ad_revenue_usd,sessions,playtime
0,2022-04-09,0.1.75.1,SG,2022-04-09,bytedanceglobal_int,EM_AND_SEA_Purchase_IT2_0604,1649548002569-6720420720726321605,False,0,0.0,0.000000,1,4
1,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649545238733-4551347935758619924,False,0,0.0,0.000000,1,2
2,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649540322226-5366969563919899948,False,0,0.0,0.000000,1,2
3,2022-04-09,0.1.75.1,UK,2022-04-07,bytedanceglobal_int,EM_AND_T1_Purchase_IT2_0604,1649337475698-617904100123526593,False,0,0.0,0.000000,1,1
4,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649544985557-9160306254813664580,False,0,0.0,0.000000,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
690370,2022-07-31,NaN,PH,2022-07-28,organic,organic,1659003289354-9130797695840500796,True,0,0.0,0.002614,1,1
690371,2022-07-31,NaN,AE,2022-07-28,organic,organic,1659020308480-7169513513931602186,True,0,0.0,0.012164,1,1
690372,2022-07-31,NaN,DZ,2022-07-29,organic,organic,1659059933950-196412188438371439,True,0,0.0,0.000195,1,1
690373,2022-07-31,NaN,FR,2022-07-29,organic,organic,1659086493021-6361713490086303799,True,0,0.0,0.021129,1,1


In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])
df.dtypes

event_date        datetime64[ns]
app_version               object
country_code              object
install_date      datetime64[ns]
media_source              object
campaign_name             object
appsflyer_id              object
is_organic                  bool
payments                   int64
revenue_usd              float64
ad_revenue_usd           float64
sessions                   int64
playtime                   int64
dtype: object

Рассмотрим расчет и анализ метрик вовлеченности пользователя - числа и длины сессий.

Посмотрим значения этих метрик для рассматриваемого проекта в 0ой день активности.

In [ ]:
df_day0 = df[df['event_date'] == df['install_date']][['appsflyer_id', 'country_code', 'is_organic', 'sessions', 'playtime']]
df_day0[['sessions', 'playtime']].mean()

sessions     1.243130
playtime    25.647484
dtype: float64

Посмотрим на медиану тех же метрик

In [ ]:
df_day0[['sessions', 'playtime']].median()

sessions     1.0
playtime    11.0
dtype: float64

Почему значения различаются на ваш взгляд?

Посмотрим на характеристики вовлеченности пользователей в 0ой день активности в разрезе канала привлечения (органика/траффик).

In [ ]:
df_day0_ch = df_day0.groupby('is_organic').agg({'sessions':'mean', 'playtime':'median'})
df_day0_ch

,sessions,playtime
is_organic,,
False,1.187394,9.0
True,1.247034,11.0


Теперь посмотрим в разрезе гео.

In [ ]:
df_day0_geo = df_day0.groupby('country_code', as_index=False).agg({'appsflyer_id':'nunique', 'sessions':'mean', 'playtime':'median'}).rename(columns = {'appsflyer_id':'users'})
df_day0_geo.sort_values(by = 'users', ascending=False).head(10)

,country_code,users,sessions,playtime
214,US,48866,1.245590,11.0
163,PH,30103,1.329880,22.0
28,BR,29332,1.154156,3.0
92,ID,16720,1.340071,21.0
147,MY,13499,1.277395,17.0
69,FR,11732,1.187489,5.0
52,DE,10241,1.225347,10.0
212,UK,9864,1.208511,11.0
100,IT,7390,1.266576,11.0
175,RU,7254,1.125689,2.0


Какие выводы можем сделать по полученным результатам? Сравните значения для отдельных гео со средними значениями для проекта.

Довольно полезно строить распределение числа пользователей по длине сессии. Построим такое распределение для 0го дня активности.

In [ ]:
df_day0['session_length'] = round(df_day0['playtime']/df_day0['sessions'])
df_day0['length_tier'] = np.where(df_day0['session_length']==0, '0', 'unknown')
df_day0['length_tier'] = np.where((df_day0['session_length']>0) & (df_day0['session_length']<=5), '1-5', df_day0['length_tier'])
df_day0['length_tier'] = np.where((df_day0['session_length']>5) & (df_day0['session_length']<=10), '5-10', df_day0['length_tier'])
df_day0['length_tier'] = np.where((df_day0['session_length']>10) & (df_day0['session_length']<=20), '10-20', df_day0['length_tier'])
df_day0['length_tier'] = np.where((df_day0['session_length']>20) & (df_day0['session_length']<=50), '20-50', df_day0['length_tier'])
df_day0['length_tier'] = np.where(df_day0['session_length']>50, '50+', df_day0['length_tier'])
df_day0

,appsflyer_id,country_code,is_organic,sessions,playtime,session_length,length_tier
0,1649548002569-6720420720726321605,SG,False,1,4,4.0,1-5
1,1649545238733-4551347935758619924,US,False,1,2,2.0,1-5
2,1649540322226-5366969563919899948,US,False,1,2,2.0,1-5
4,1649544985557-9160306254813664580,US,False,1,2,2.0,1-5
5,1649543024686-2013762249241156998,US,False,1,1,1.0,1-5
...,...,...,...,...,...,...,...
689922,1659249569110-650678553241268692,ID,True,1,1,1.0,1-5
689966,1659304893612-6320259533456109928,ES,True,1,1,1.0,1-5
690261,1659274189573-6833361210332874832,IT,True,1,1,1.0,1-5
690296,1659228138750-4796048781949470717,BR,True,1,1,1.0,1-5


In [ ]:
df_day0_distr = df_day0.groupby('length_tier', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'})

fig1 = px.bar(df_day0_distr, x='length_tier', y='users')
fig1.show()

Теперь рассмотрим lifetime метрики и сопоставим их с lifetime пользователя - для этого добавим колонку activity_day для каждого юзера

In [ ]:
df['activity_day'] = (df['event_date'] - df['install_date']).dt.days
df

,event_date,app_version,country_code,install_date,media_source,campaign_name,appsflyer_id,is_organic,payments,revenue_usd,ad_revenue_usd,sessions,playtime,activity_day
0,2022-04-09,0.1.75.1,SG,2022-04-09,bytedanceglobal_int,EM_AND_SEA_Purchase_IT2_0604,1649548002569-6720420720726321605,False,0,0.0,0.000000,1,4,0
1,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649545238733-4551347935758619924,False,0,0.0,0.000000,1,2,0
2,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649540322226-5366969563919899948,False,0,0.0,0.000000,1,2,0
3,2022-04-09,0.1.75.1,UK,2022-04-07,bytedanceglobal_int,EM_AND_T1_Purchase_IT2_0604,1649337475698-617904100123526593,False,0,0.0,0.000000,1,1,2
4,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649544985557-9160306254813664580,False,0,0.0,0.000000,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690370,2022-07-31,NaN,PH,2022-07-28,organic,organic,1659003289354-9130797695840500796,True,0,0.0,0.002614,1,1,3
690371,2022-07-31,NaN,AE,2022-07-28,organic,organic,1659020308480-7169513513931602186,True,0,0.0,0.012164,1,1,3
690372,2022-07-31,NaN,DZ,2022-07-29,organic,organic,1659059933950-196412188438371439,True,0,0.0,0.000195,1,1,2
690373,2022-07-31,NaN,FR,2022-07-29,organic,organic,1659086493021-6361713490086303799,True,0,0.0,0.021129,1,1,2


In [ ]:
df_lt = df.groupby(['appsflyer_id', 'is_organic', 'country_code'], as_index=False).agg({'sessions':'sum', 'playtime':'sum', 'activity_day':'max'}).rename(columns = {'activity_day':'lifetime'})
df_lt

,appsflyer_id,is_organic,country_code,sessions,playtime,lifetime
0,1262310748796-5356552856195451152,True,CO,5,3,34
1,1483230136294-9150424549124065990,True,BR,1,1,0
2,1638582970315-7825904685091974346,True,DE,1,19,137
3,1638619343502-8901026392180839604,True,US,15,331,213
4,1638622210305-5317933551179393503,True,FR,3,28,288
...,...,...,...,...,...,...
335190,1659617639772-3599033371225115285,True,GR,2,2,1
335191,1659786741876-641409135904582475,True,BR,1,1,0
335192,1659838019758-8805130209974056605,True,CR,1,30,0
335193,1659945407863-1685429817144280172,True,PH,1,53,0


Посмотрим топ игроков, которые наиграли больше всего игрового времени за лайфтайм

In [ ]:
df_lt.sort_values(by = 'playtime', ascending = False).head(15)

,appsflyer_id,is_organic,country_code,sessions,playtime,lifetime
181577,1656356789050-5702558459993875961,True,SE,61,1737,34
114428,1655236086225-5352649855997769788,True,BR,37,1670,47
129638,1655564250067-3308286963051170480,True,BR,32,1620,36
7565,1649549398698-8461308303697459509,True,US,119,1567,112
114712,1655242195423-1789234642210469775,True,SA,61,1536,40
47789,1652919125350-3314503496059404302,True,US,56,1515,71
10425,1649944023400-4483360233886984342,True,CA,61,1496,39
258117,1657499520896-5459486256796620578,True,BR,30,1483,20
129672,1655564780561-7113308472061074685,True,BR,41,1439,34
102207,1654939170346-8888642778746252254,True,US,27,1417,38


Какие выводы можем сделать?

Посмотрим распределение метрик по гео.

In [ ]:
df_lt_geo = df_lt.groupby('country_code', as_index=False).agg({'appsflyer_id':'nunique', 'sessions':'sum', 'playtime':'sum'}).rename(columns = {'appsflyer_id':'users'})
df_lt_geo = df_lt_geo.sort_values(by = 'users', ascending=False).head(10)

df_lt_geo['sessions_per_user'] = df_lt_geo['sessions'] / df_lt_geo['users']
df_lt_geo['playtime_per_user'] = df_lt_geo['playtime'] / df_lt_geo['users']

df_lt_geo

,country_code,users,sessions,playtime,sessions_per_user,playtime_per_user
214,US,50652,135863,2517206,2.682283,49.696083
163,PH,30911,90238,2013880,2.919284,65.150917
28,BR,29914,62463,1046514,2.088086,34.984088
92,ID,17077,50571,1046551,2.961352,61.284242
147,MY,14038,37517,684921,2.672532,48.790497
69,FR,12101,32499,524836,2.685646,43.371292
52,DE,10832,28411,536287,2.622877,49.509509
212,UK,10176,26372,486373,2.591588,47.796089
100,IT,7557,22236,420276,2.942437,55.614133
175,RU,7412,13941,213977,1.880869,28.868996


In [ ]:
fig2 = px.bar(df_lt_geo, x = 'country_code', y='sessions_per_user')
fig2.show()

In [ ]:
fig3 = px.bar(df_lt_geo, x='country_code', y='playtime_per_user')
fig3.show()

Какие выводы можно сделать по полученным результатам?

# Домашнее задание

Проанализируйте метрики вовлеченности пользователей по различным когортам (это могут каналы привлечения, гео, временные периоды, платящие/не платящие игроки и т.д.). Какая когорта показывает наилучшую вовлеченность в игру?

Количество ссесий в день установки 1,24, среднее время игры 25,6 минут. Медиана сессии на 1 играка составляет 11 минут. Это значит, что есть игроки который могут длительно играть, но средний показатель 11 минут. В этом случае результат не очень хороший, так как игрок зашел всего 1 раз за день на небольшое время.

Играки из США играют медианную сессию, при этом от туда самое большое количество игроков. В Филлипинах и Индонезии время увеличенно в 2  раза по этношению к стандартному. В России, Бразилии и Франции игровое время самое низкое. Это показывает целевую аудиторию игры.

Из-за большого количества игроков, около 30000, находившихся в игре меньше минуты. свидетельствует о возможности ошибки. Необходимо выяснить регион и период игры в который это происходит.